In [29]:
!pip install azure-cognitiveservices-vision-computervision azure-ai-textanalytics pillow


In [30]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from msrest.authentication import CognitiveServicesCredentials
from PIL import Image
 
# Add your Computer Vision subscription key and endpoint
subscription_key = ""
endpoint = "https://openaicomvision.cognitiveservices.azure.com/"
 
# Create a client
computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))

In [31]:
import io
 
# Open an image file
image_path = "handwriten_french.jpg"
with open(image_path, "rb") as image_stream:
    # Perform OCR (Optical Character Recognition) on the image
    ocr_result = computervision_client.read_in_stream(image_stream, raw=True)
 
# Extract text from the result
operation_location = ocr_result.headers["Operation-Location"]
operation_id = operation_location.split("/")[-1]
 
# Get the result
result = computervision_client.get_read_result(operation_id)
while result.status not in ["succeeded", "failed"]:
    result = computervision_client.get_read_result(operation_id)
 
# Extract lines of text
lines = []
if result.status == "succeeded":
    for text_result in result.analyze_result.read_results:
        for line in text_result.lines:
            lines.append(line.text)
 
# Join lines to form the complete text
handwritten_text = "\n".join(lines)
print(handwritten_text)

Old French School
J'ai voulu approcher l'écriture telle qu'on la
pratiquait encore dans les années 60 dans mon
école primaire à Suilly la Tour ( Nievre). La plume
était toujours obligatoire en classe même si le style
à bille s'était déjà imposé partout ailleurs. Il faudra
le tourbillon de 1968 pour la faire disparaître avec
les taches, les pages arrachées et les doigts violets.


In [32]:
from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential
 
# Add your Text Analytics subscription key and endpoint
text_analytics_key = ""
text_analytics_endpoint = "https://langtrain123.cognitiveservices.azure.com/"
 
# Create a client
text_analytics_client = TextAnalyticsClient(endpoint=text_analytics_endpoint, credential=AzureKeyCredential(text_analytics_key))

In [33]:
!pip install requests

In [34]:
import requests
import json
 
# Azure Translator credentials
translator_key = ""
translator_endpoint = "https://api.cognitive.microsofttranslator.com/"
translator_location = "eastus"
 
# Function to translate text
def translate_text(text, target_language="en"):
    path = '/translate'
    constructed_url = translator_endpoint + path
 
    headers = {
        'Ocp-Apim-Subscription-Key': translator_key,
        'Ocp-Apim-Subscription-Region': translator_location,
        'Content-type': 'application/json'
    }
    params = {
        'api-version': '3.0',
        'to': target_language
    }
    body = [{
        'text': text
    }]
    request = requests.post(constructed_url, params=params, headers=headers, json=body)
    response = request.json()
    return response[0]['translations'][0]['text']

In [35]:
# Example: Sentiment Analysis
def analyze_text(text):
    response = text_analytics_client.analyze_sentiment(documents=[text])[0]
    print(f"Sentiment: {response.sentiment}")
    print(f"Positive score: {response.confidence_scores.positive}")
    print(f"Neutral score: {response.confidence_scores.neutral}")
    print(f"Negative score: {response.confidence_scores.negative}")
 
# Example: Key Phrase Extraction
def extract_key_phrases(text):
    response = text_analytics_client.extract_key_phrases(documents=[text])[0]
    print("Key Phrases:")
    for phrase in response.key_phrases:
        print(f"- {phrase}")
def detect_language(text):
    response = text_analytics_client.detect_language(documents=[text])[0]
    print(f"\nDetected Language: {response.primary_language.name}")
    print(f"ISO 6391 Code: {response.primary_language.iso6391_name}")
    print(f"Confidence Score: {response.primary_language.confidence_score}")

# Detect language of the handwritten text
detect_language(handwritten_text)


# Analyze the handwritten text
analyze_text(handwritten_text)
extract_key_phrases(handwritten_text)

if detect_language != "en":
    translated_text = translate_text(handwritten_text, "en")
    print(f"\nTranslated Text:\n{translated_text}")
else:
    translated_text = handwritten_text
 
# Perform sentiment analysis and key phrase extraction on the translated text
analyze_text(translated_text)
extract_key_phrases(translated_text)


Detected Language: French
ISO 6391 Code: fr
Confidence Score: 1.0
Sentiment: mixed
Positive score: 0.33
Neutral score: 0.38
Negative score: 0.29
Key Phrases:
- Old French School
- Suilly la Tour
- La plume
- années
- école primaire
- doigts violets
- écriture
- Nievre
- classe
- style
- bille
- tourbillon
- taches
- pages
- 60

Translated Text:
Old French School
I wanted to approach writing as it is
was still practicing in the 60s in my
primary school at Suilly la Tour (Nievre). The pen
was still mandatory in the classroom even though the style
had already established itself everywhere else. It will be necessary
tourbillon of 1968 to make it disappear with
the spots, the torn pages and the purple fingers.
Sentiment: negative
Positive score: 0.01
Neutral score: 0.27
Negative score: 0.72
Key Phrases:
- Suilly la Tour
- Old French School
- primary school
- torn pages
- purple fingers
- writing
- 60s
- Nievre
- pen
- classroom
- style
- tourbillon
- spots
